In [82]:
%pylab inline
import pandas as pd
import numpy as np
from statsmodels.tools import categorical

Populating the interactive namespace from numpy and matplotlib


# Data


[Kaggle Titanic Data](https://www.kaggle.com/c/titanic/data)에서 자세한 데이터 정보를 얻을수 있습니다.

In [168]:
# Train Data
train = pd.read_csv('../../data/titanic/train.csv', usecols=(1, 2, 4, 5, 6, 7, 9, 11))
train['Survived'] = train['Survived'].factorize()[0]
train['Pclass'] = train['Pclass'].factorize()[0]
train['Sex'] = train['Sex'].factorize()[0]
train['Embarked'] = train['Embarked'].factorize()[0]

train_x = train.drop('Survived', axis=1).as_matrix()
train_y = train['Survived'].as_matrix()


# Test Data
test = pd.read_csv('../../data/titanic/test.csv', usecols=(0, 1, 3, 4, 5, 6, 8, 10))
test = pd.merge(test, pd.read_csv('../../data/titanic/gendermodel.csv'), on='PassengerId')
test = test.drop('PassengerId', axis=1)

test['Survived'] = test['Survived'].astype('category')
test['Pclass'] = test['Pclass'].astype('category')
test['Sex'] = test['Sex'].astype('category')
test['Embarked'] = test['Embarked'].astype('category')

test_x = test.drop('Survived', axis=1).as_matrix()
test_y = test['Survived'].as_matrix()

print '[Training Data]'
print train.head()
print

print '[Test Data]'
print train.dtypes

[[  0.       0.      22.     ...,   0.       7.25     0.    ]
 [  1.       1.      38.     ...,   0.      71.2833   1.    ]
 [  0.       1.      26.     ...,   0.       7.925    0.    ]
 ..., 
 [  0.       1.          nan ...,   2.      23.45     0.    ]
 [  1.       0.      26.     ...,   0.      30.       1.    ]
 [  0.       0.      32.     ...,   0.       7.75     2.    ]]
[Training Data]
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0         0       0    0  22.0      1      0   7.2500         0
1         1       1    1  38.0      1      0  71.2833         1
2         1       0    1  26.0      0      0   7.9250         0
3         1       1    1  35.0      1      0  53.1000         0
4         0       0    0  35.0      0      0   8.0500         0

[Test Data]
Survived      int64
Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked      int64
dtype: object


In [211]:
d = np.nanstd(train_x, axis=0)
N, C = train_x.shape
x = [len(np.unique(train_x[:, i]))/float(N) for i in xrange(C)]
print x < np.mean(x)
print d < 3

[ True  True False  True  True False  True]
[ True  True False  True  True False  True]


# Decision Tree with Python

In [249]:
class DecisionTree(object):
    
    def __init__(self, impurity='error'):
        assert impurity in ('entropy', 'gini', 'error')
        self.impurity = getattr(self, impurity)
        
    def prepare_data(self, data):
        self.categories = np.nanstd(data, axis=0) < 3
    
    def train(self, X, Y):
        self.prepare_data(X)
        
        C = X.shape[1]
        for i, is_categorical in enumerate(self.categories):
            if is_categorical:
                self.information_gain(X[:, i], X[:, i], Y)
            
        
    def information_gain(self, p, x, y):
        Dp = self.impurity(p)
        
    def entropy(self, p):
        return -(p*np.log2(p) + (1-p)*np.log2(1-p))
    
    def gini(self, p):
        return 1 - (p**2 + (1-p)**2)
    
    def error(self, data):
        shapes = [float(data[data == c].size) for c in np.unique(data)]
        shapes = shapes/np.sum(shapes)
        return 1 - np.max(shapes)
        
        
#         return 1 - np.max([p, 1-p], axis=0)
    
dt = DecisionTree()   
dt.train(train_x, train_y)

0.448933782267
0.35241301908
0.317620650954
0.239057239057
0.27721661055
